In [1]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation, Bidirectional
from tensorflow.keras.optimizers import RMSprop

## Importing the file

In [2]:
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
text = open(filepath, 'rb').read().decode(encoding = 'utf-8').lower()

## Preprocessing the data

In [5]:
len(text)

1115394

In [6]:
text[:10000]

"first citizen:\nbefore we proceed any further, hear me speak.\n\nall:\nspeak, speak.\n\nfirst citizen:\nyou are all resolved rather to die than to famish?\n\nall:\nresolved. resolved.\n\nfirst citizen:\nfirst, you know caius marcius is chief enemy to the people.\n\nall:\nwe know't, we know't.\n\nfirst citizen:\nlet us kill him, and we'll have corn at our own price.\nis't a verdict?\n\nall:\nno more talking on't; let it be done: away, away!\n\nsecond citizen:\none word, good citizens.\n\nfirst citizen:\nwe are accounted poor citizens, the patricians good.\nwhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them let us revenge this with\nour pikes, ere we become rakes: for the gods know i\nspeak this in hunger 

In [19]:
# getting all the individual text in text
characters = sorted(set(text))

In [20]:
characters

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [63]:
# we need an dictionary so that we can convert associate a numeric value with a character

index_to_char = dict((char, index) for char, index in enumerate(characters))

In [40]:
index_to_char

{0: '\n',
 1: ' ',
 2: '!',
 3: '$',
 4: '&',
 5: "'",
 6: ',',
 7: '-',
 8: '.',
 9: '3',
 10: ':',
 11: ';',
 12: '?',
 13: 'a',
 14: 'b',
 15: 'c',
 16: 'd',
 17: 'e',
 18: 'f',
 19: 'g',
 20: 'h',
 21: 'i',
 22: 'j',
 23: 'k',
 24: 'l',
 25: 'm',
 26: 'n',
 27: 'o',
 28: 'p',
 29: 'q',
 30: 'r',
 31: 's',
 32: 't',
 33: 'u',
 34: 'v',
 35: 'w',
 36: 'x',
 37: 'y',
 38: 'z'}

In [62]:
# we need a dictionary to convert back the values of characters to the respective index 

char_to_index = dict((index, char) for char, index in enumerate(characters))

In [42]:
char_to_index

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'a': 13,
 'b': 14,
 'c': 15,
 'd': 16,
 'e': 17,
 'f': 18,
 'g': 19,
 'h': 20,
 'i': 21,
 'j': 22,
 'k': 23,
 'l': 24,
 'm': 25,
 'n': 26,
 'o': 27,
 'p': 28,
 'q': 29,
 'r': 30,
 's': 31,
 't': 32,
 'u': 33,
 'v': 34,
 'w': 35,
 'x': 36,
 'y': 37,
 'z': 38}

## Next Character Prediction 

Our main goal is to feed our neural network with all the characters above in the dataset and then we want our neural network to predict the next character according to the shakespear texts. Now, we need to specify how many of our characters we want our neural network to look into before predicting the next character.

We need to decide two things : Sequence Length and step size. Sequence length is the amount of characters our neural network looks before predicting next character, and step size and how many characters we move before we consider it another sequence. 

In [28]:
SEQ_LENGTH = 50
STEP_SIZE = 3

In [30]:
sentences = [] # We store sequences of seq_length in this 
next_characters = [] # the corresponding next characters to the sequences is stored here

In [32]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i : i + SEQ_LENGTH])
    next_characters.append(text[i+SEQ_LENGTH])

### Conversion of training data from string to numeric values

For input values, we need an array which has all the sentences, and inside it, we need all the characters, and then, we need a way to uniquely identify the character. So, we make a 3-dimensional boolean array to acheive this.

For output values, we need an array with all the sentences, and the next actual character in the text to the sample sequence. We do that by using 2-dimensional boolean array.

In [36]:
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype = bool)
y = np.zeros((len(sentences), len(characters)), dtype = bool)

In [43]:
for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        x[i, t, char_to_index[character]] = True
    y[i, char_to_index[next_characters[i]]] = True

## Neural Network

We will use a simple model with LSTM as the starting layer, followed by a dense layer with softmax function in order to predict the next character.

In [46]:
model = Sequential()
model.add(LSTM(128, input_shape = (SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128)               86016     
                                                                 
 dense_1 (Dense)             (None, 39)                5031      
                                                                 
 activation (Activation)     (None, 39)                0         
                                                                 
Total params: 91047 (355.65 KB)
Trainable params: 91047 (355.65 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [49]:
# Using Legacy RMSprop as it works better on M1/M2 Macbooks, otherwise normal will do it not problem
model.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.legacy.RMSprop(learning_rate = 0.01))

In [50]:
model.fit(x, y, batch_size = 256, epochs = 10)

Epoch 1/10


2023-09-01 15:25:59.708767: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-01 15:25:59.861646: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-01 15:26:00.496070: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1453/1453 [==============================] - 31s 20ms/step - loss: 1.8944
Epoch 2/10
1453/1453 [==============================] - 29s 20ms/step - loss: 1.5675
Epoch 3/10
1453/1453 [==============================] - 29s 20ms/step - loss: 1.4949
Epoch 4/10
1453/1453 [==============================] - 29s 20ms/step - loss: 1.4568
Epoch 5/10
1453/1453 [==============================] - 29s 20ms/step - loss: 1.4314
Epoch 6/10
1453/1453 [==============================] - 29s 20ms/step - loss: 1.4151
Epoch 7/10
1453/1453 [==============================] - 28s 20ms/step - loss: 1.4008
Epoch 8/10
1453/1453 [==============================] - 29s 20ms/step - loss: 1.3911
Epoch 9/10
1453/1453 [==============================] - 28s 19ms/step - loss: 1.3826
Epoch 10/10
1453/1453 [==============================] - 28s 19ms/step - loss: 1.3759


In [51]:
model.save('shakespearTextGenerator.model')

INFO:tensorflow:Assets written to: shakespearTextGenerator.model/assets


INFO:tensorflow:Assets written to: shakespearTextGenerator.model/assets


## Generating Sonnets

From our model, we will get a softmax return with probability of the next character from list of all characters. We need to use this to create actual samples of text. So, we take a sample text from our dataset, then, we use that text to predict the next value of our model. We use something called temprature in order to access the risk associated with the word we are going to choose. The more the temprature, the more the risk of choosing the incorrect character. We do that with the following two functions, and generate text like shakespear.

In [52]:
def sample(preds, temprature = 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temprature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [56]:
def generate_text(length, temprature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated_text = ''
    sentence = text[start_index : start_index + SEQ_LENGTH]
    generated_text += sentence
    for i in range(length):
        x = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, character in enumerate(sentence):
            x[0, t, char_to_index[character]] = 1

        predictions = model.predict(x, verbose = 0)[0]
        next_index = sample(predictions, temprature)
        generated_text += index_to_char[next_index]
        sentence = sentence[1:] + index_to_char[next_index]

    return generated_text

In [57]:
generate_text(500, 0.2)

2023-09-01 15:59:03.253981: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-01 15:59:03.326180: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


'joyful to hear of their readiness, and am the\nman, and the store of the shall be a lives of him.\n\nantonio:\nthe shall be a lives of the sisters of the world\nto the world and stand the prince and the men\nthat is the seasold and soul the prince\nand the seasoness to be a life,\nand the sechine and the sunder that is the country\nand see the state to the mighty state,\nand the rest of the shall shall be a field\nto see thy life and the store of the duke\nand the sun that i will have done the world,\nand the worst of the world that with me,\nthat i have not'

In [58]:
generate_text(500, 0.4)

' all our loss again;\nthe queen from france hath brother than the horse:\nthe world comes to the sun and have have a\nfither the gods the countenance and this the death;\nthat the king of a store, whose hand the prince\nand which we are the world, and sword henry.\n\ngrumio:\npro?kers have heard thee, he is a brands,\nto learn the gates and seem the best of the light,\nthe leor for the sad and store, which the prehender\nto be man the dead flower man of suffered\nto and the best of the cause the soul to the face\nthe gods and done to hear thee, and be hence'

In [59]:
generate_text(500, 0.6)

'ave crafted fair!\n\ncominius:\nyou have brought\na troop the king of the bring and prince\nthe sin of the heart be the states have to have\nthe time; if i inted the mercy to lessed\nthe seemings and his life, in the dead?\n\nclaudio:\nif thee thought my tented a father to the cause\nto atting strice upon the villain to drift\nthe very treath, the sist than i thank itself\nthe for his sish the sister time to the soul of my country\nas is dead, i will not to thee;\npetruchio that that have adam me to himself\nthis is the for his bowbreas the shast to great off\n'

In [60]:
generate_text(500, 0.8)

"ued, are but light to me,\nmight i but through my pass, that heard thee,\nwhen whilst his now,\nand not say, have straugh name,\nthe call; the suffrcond and sight.\n\nmarcius:\nmy gentleman, my corsins mornast.\n\nlord:\nnow, hart thy derises the court:\nand thou diss'dial montague,\nand tender this villain will be a strong.\nwhat's the hound and tell thee.\n\nwarwick:\ni have done: though henry shrie are.\n\nuliblet:\nsay it not shall be me which here wegge he was\nthy love live in isable; be such moge\nshall be walce should his mine and there which grown,\nand so "

In [61]:
generate_text(500, 1)

"you have no cause.\n\narchbishop of york:\nmy gracious liege:\nwhilst who from cleation of dong to take\nmy none; if you have full fair as if your joked awely:\ndone all some laid that my nisemser which duke\nwith such in feataboly of this commer.\nwhat, what, fair, beshorg i do wad brother,\nthis the urpilawidh\nto this time no magammposs'd, and them peace.\n\ncapulet:\nmadam, have speak age is from thy lord thou\nart distorm is ear is a liberclainn,\nthe plust done, and tuntrourle-house, his wall:\nis patserty, you stoot tendor,\nyou troophined; and warwick h"

Overall, I think model is a success. Since LLMs are really good at identifying patterns, maybe we could tune an LLM to predict shakespear text like this.